In [1]:
import os
import json

# --- Modify these variables directly to change inputs ---
evaluation_llm_dir = "evaluations_llm"
evaluation_rouge_dir = "evaluations_rouge"
exp_name = "none-1B"
communication_rounds = "20"
rouge_metric = "rouge1"

# --- Domain mapping for both functions ---
domains = {
    'BS': 'brainstorming',
    'CL': 'classification',
    'CQ': 'closed_qa',
    'CW': 'creative_writing',
    'GQ': 'general_qa',
    'IE': 'information_extraction',
    'OQ': 'open_qa',
    'SM': 'summarization'
}

In [2]:
def compute_statistics(results):
    """Compute row/column averages and overall average"""
    n = len(results)
    
    # Row averages
    row_avgs = []
    for row in results:
        vals = [v for v in row if v is not None]
        row_avgs.append(sum(vals)/len(vals) if vals else None)
    
    # Column averages
    col_avgs = []
    for j in range(n):
        vals = [results[i][j] for i in range(n) if results[i][j] is not None]
        col_avgs.append(sum(vals)/len(vals) if vals else None)
    
    # Overall average
    overall_avg = sum(a for a in row_avgs if a is not None) / len([a for a in row_avgs if a is not None])
    
    return row_avgs, col_avgs, overall_avg

def generate_latex_table_llm(evaluation_dir, exp_name, communication_rounds):
    """Generate LaTeX table for LLM evaluation (min is better)"""
    n = len(domains)
    
    # Load results into an n×n matrix
    results = [[None]*n for _ in range(n)]
    base_path = os.path.join(evaluation_dir, exp_name, communication_rounds)
    for i in range(n):
        filepath = os.path.join(base_path, f"client_{i}_results.json")
        try:
            data = json.load(open(filepath))
            for j, abbr in enumerate(domains):
                key = domains[abbr]
                if key in data['categories']:
                    results[i][j] = data['categories'][key]['ratio']
        except Exception as e:
            print(f"Warning for client {i}: {e}")
    
    # Compute statistics
    row_avgs, col_avgs, overall_avg = compute_statistics(results)
    
    # Identify minimum values for bolding
    col_min = [min(results[i][j] or float('inf') for i in range(n) if results[i][j] is not None) for j in range(n)]
    min_row_avg = min(a or float('inf') for a in row_avgs if a is not None)
    
    # Build LaTeX table
    lines = []
    lines.append(r"\begin{tabular}{l l|cccccccc|c}")
    lines.append(r"\hline")
    lines.append(r"\textbf{Method} & \textbf{Training} & \multicolumn{8}{c|}{\textbf{Test Domain}} & \textbf{Avg.} \\")
    lines.append(r" & \textbf{Domain} & \textbf{BS} & \textbf{CL} & \textbf{CQ} & \textbf{CW} & \textbf{GQ} & \textbf{IE} & \textbf{OQ} & \textbf{SM} & \\")
    lines.append(r"\hline")
    lines.append(r"\multirow{%d}{*}{Individual}" % n)
    
    # Data rows
    for i in range(n):
        row = ["", list(domains.keys())[i]]
        for j in range(n):
            v = results[i][j]
            if v is None:
                cell = "--"
            else:
                text = f"{v:.3f}"
                if v == col_min[j]:
                    text = r"\textbf{" + text + "}"
                cell = text
            row.append(cell)
        
        avg = row_avgs[i]
        avg_text = f"{avg:.3f}" if avg is not None else "--"
        if avg == min_row_avg:
            avg_text = r"\textbf{" + avg_text + "}"
        row.append(avg_text)
        lines.append(" & ".join(row) + r" \\")
    
    # Column-average row
    lines.append(r"\hline")
    avg_cells = [f"{v:.3f}" for v in (col_avgs + [overall_avg])]
    lines.append(
        r"\multicolumn{2}{c|}{\textbf{Average}} & "
        + " & ".join(avg_cells)
        + r" \\"
    )
    
    lines.append(r"\hline")
    lines.append(r"\end{tabular}")
    return "\n".join(lines)

def generate_latex_table_rouge(evaluation_dir, exp_name, communication_rounds, rouge_metric):
    """Generate LaTeX table for ROUGE evaluation (max is better)"""
    n = len(domains)
    
    # Load results into an n×n matrix
    results = [[None]*n for _ in range(n)]
    base_path = os.path.join(evaluation_dir, exp_name, communication_rounds)
    
    for i in range(n):
        filepath = os.path.join(base_path, f"client_{i}_output_rouge.json")
        try:
            data = json.load(open(filepath))
            for j, abbr in enumerate(domains):
                key = domains[abbr]
                if key in data:
                    results[i][j] = data[key][rouge_metric]
        except Exception as e:
            print(f"Warning for client {i}: {e}")
    
    # Compute statistics
    row_avgs, col_avgs, overall_avg = compute_statistics(results)
    
    # Identify maximum values for bolding
    col_max = [max(results[i][j] or 0 for i in range(n) if results[i][j] is not None) for j in range(n)]
    max_row_avg = max(a or 0 for a in row_avgs if a is not None)
    
    # Build LaTeX table
    lines = []
    lines.append(r"\begin{tabular}{l l|cccccccc|c}")
    lines.append(r"\hline")
    lines.append(r"\textbf{Method} & \textbf{Training} & \multicolumn{8}{c|}{\textbf{Test Domain}} & \textbf{Avg.} \\")
    lines.append(r" & \textbf{Domain} & \textbf{BS} & \textbf{CL} & \textbf{CQ} & \textbf{CW} & \textbf{GQ} & \textbf{IE} & \textbf{OQ} & \textbf{SM} & \\")
    lines.append(r"\hline")
    lines.append(r"\multirow{%d}{*}{Individual}" % n)
    
    # Data rows
    for i in range(n):
        row = ["", list(domains.keys())[i]]
        for j in range(n):
            v = results[i][j]
            if v is None:
                cell = "--"
            else:
                text = f"{v:.3f}"
                if v == col_max[j]:  # Bold the maximum value in each column
                    text = r"\textbf{" + text + "}"
                cell = text
            row.append(cell)
        
        avg = row_avgs[i]
        avg_text = f"{avg:.3f}" if avg is not None else "--"
        if avg == max_row_avg:  # Bold the maximum row average
            avg_text = r"\textbf{" + avg_text + "}"
        row.append(avg_text)
        lines.append(" & ".join(row) + r" \\")
    
    # Column-average row
    lines.append(r"\hline")
    avg_cells = [f"{v:.3f}" for v in (col_avgs + [overall_avg])]
    lines.append(
        r"\multicolumn{2}{c|}{\textbf{Average}} & "
        + " & ".join(avg_cells)
        + r" \\"
    )
    
    lines.append(r"\hline")
    lines.append(r"\end{tabular}")
    return "\n".join(lines)

In [3]:
# Generate and print tables
latex_code_llm = generate_latex_table_llm(evaluation_llm_dir, exp_name, communication_rounds)
print(latex_code_llm)
print("\n")

latex_code_rouge = generate_latex_table_rouge(evaluation_rouge_dir, exp_name, communication_rounds, rouge_metric)
print(latex_code_rouge)

\begin{tabular}{l l|cccccccc|c}
\hline
\textbf{Method} & \textbf{Training} & \multicolumn{8}{c|}{\textbf{Test Domain}} & \textbf{Avg.} \\
 & \textbf{Domain} & \textbf{BS} & \textbf{CL} & \textbf{CQ} & \textbf{CW} & \textbf{GQ} & \textbf{IE} & \textbf{OQ} & \textbf{SM} & \\
\hline
\multirow{8}{*}{Individual}
 & BS & 1.034 & 1.165 & 1.519 & 1.097 & 1.182 & 2.190 & 1.121 & 1.556 & 1.358 \\
 & CL & 1.108 & \textbf{1.094} & 1.304 & 1.188 & 1.120 & 1.783 & 1.201 & 1.361 & 1.270 \\
 & CQ & 1.267 & 1.189 & 1.319 & 1.266 & 1.298 & 1.913 & 1.255 & 1.472 & 1.372 \\
 & CW & \textbf{0.957} & 1.227 & 1.305 & \textbf{1.065} & \textbf{0.991} & 1.495 & 1.019 & 1.360 & \textbf{1.177} \\
 & GQ & 1.063 & 1.456 & 1.269 & 1.183 & 1.059 & 1.444 & \textbf{1.011} & 1.320 & 1.226 \\
 & IE & 1.206 & 1.533 & \textbf{1.088} & 1.243 & 1.166 & 1.384 & 1.303 & 1.182 & 1.263 \\
 & OQ & 1.137 & 1.563 & 1.223 & 1.265 & 1.203 & 1.420 & 1.277 & 1.299 & 1.298 \\
 & SM & 1.137 & 1.436 & 1.143 & 1.283 & 1.172 & \textbf{1.252